# 2.1 Goldman-Sachs 기반 Valuation
last updated : 2023-06-30

Goldman Sachs 모형을 활용한 CB 및 RCPS Valaution 을 수행한다.
단,본실습은 설명의 편의상 앞선 엑셀실습과 같이 노드기간, 이자율 등의 변수에 대한 가정을 단순하게 정의하였다.

### input 정의
- s : 기초자산
- rf : 무위험이자율
- kd : 위험이자율
- volatility : 변동성
- t : 만기
- x : 행사가액
- put_ytm : put만기수익률
- u : 상승확률
- d : 하락확률
- p : 위험중립확률 ( p )
- q : 위험중립확률 ( q )

In [1]:
import numpy as np

node_count = 11
s = 100000
x = 100000
put_yield = 0.05
rf = 0.015 # 무위험이자율
kd = 0.05 # 위험이자율 
volatility = 0.255
delta_t = 1
u = np.exp(volatility* np.sqrt(delta_t)) 
d = 1 / u
p = (np.exp(rf * delta_t) - d) / (u - d)
q = 1 - p

![image.png](attachment:image.png)

### Step 1: 날짜 배열 생성

In [2]:
import numpy as np
import pandas as pd

# 편의상 노드는 현재시점 ~ 10년까지 총 11개 노드로 구성

node_array = arr = np.arange(0, node_count)
print(node_array)
# 출력 : [ 0  1  2  3  4  5  6  7  8  9 10]

[ 0  1  2  3  4  5  6  7  8  9 10]


### Step 2: 부트스트래핑을 사용하여 kd 및 rf spot rate 및 forward rate 계산

In [3]:
rf_forward_array = rf + 0.001 * node_array
kd_forward_array = kd + 0.0001 * node_array

In [4]:
rf_forward_array
print(rf_forward_array)
# 출력 : [0.015, 0.016, 0.017, 0.018, 0.019, 0.02 , 0.021, 0.022, 0.023,0.024]


[0.015 0.016 0.017 0.018 0.019 0.02  0.021 0.022 0.023 0.024 0.025]


In [5]:
kd_forward_array
print(kd_forward_array)
# 출력 : [0.05  0.051 0.052 0.053 0.054 0.055 0.056 0.057 0.058 0.059]

[0.05   0.0501 0.0502 0.0503 0.0504 0.0505 0.0506 0.0507 0.0508 0.0509
 0.051 ]


### Step 3: 상환전환우선주 상환금액 트리 생성 (주계약/put)

In [6]:
# 행사가액 x ( 1 + put 이자율) ^ 각 시점별 제곱
put_ammount_array = x * np.power((1+put_yield), node_array)

In [7]:
print(put_ammount_array)
# 출력 : 
"""
[100000.         105000.         110250.         115762.5
 121550.625      127628.15625    134009.5640625  140710.04226563
 147745.54437891 155132.82159785 162889.46267774]
"""

[100000.         105000.         110250.         115762.5
 121550.625      127628.15625    134009.5640625  140710.04226563
 147745.54437891 155132.82159785 162889.46267774]


'\n[100000.         105000.         110250.         115762.5\n 121550.625      127628.15625    134009.5640625  140710.04226563\n 147745.54437891 155132.82159785 162889.46267774]\n'

In [8]:
# 가장 뒤에있는 상환가능금액에서 해당 시점에 해당하는 kd_forward_rate를 고려하여 상환가능금액에서 채권의 가치를 산출합니다.


# put_ammount_array와 동일한 크기의 bond_array를 만든다.
bond_array = np.zeros_like(put_ammount_array)
# 가장 마지막 가치는 bond_array와 put_ammount_array가 동일합니다. 
bond_array[-1] = put_ammount_array[-1]

# 마지막번째에서 1개 노드 하나 앞에부터 for-loop 생성
for i in reversed(range(node_count-1)):
    bond_array[i] = bond_array[i+1] / (1 + kd_forward_array[i+1])

In [9]:
bond_array

array([ 99477.73357059, 104461.56802248, 109705.53873721, 115223.72733569,
       121031.00319341, 127143.06885468, 133576.50813872, 140348.83710136,
       147478.5580261 , 154985.21662963, 162889.46267774])

### Step 4: 주가 이항 트리 생성


In [10]:
# 노드개수만큼의 0으로 이루어진, n x n 행렬생성 
stock_binomial_tree = np.zeros((node_count, node_count))

# 가장 첫번째 노드에 기초자산주가 설정
stock_binomial_tree[0, 0] = s

# 2번째 노드부터 마지막 노드까지 순차적으로 실행되는 for-loop 실행
for i in range(1, node_count):
    
    # 상승노드의 경우, 상승계수만 반영
    stock_binomial_tree[0, i] = stock_binomial_tree[0, i-1] * u
    
    # 하락노드
    stock_binomial_tree[1:i+1, i] = stock_binomial_tree[:i, i-1] * d


In [11]:
# 11 x 11 배열 중, 앞 3 x 3 배열만 보여줌
print(stock_binomial_tree[:3, :3])

# 출력 : 
"""
[[100000.         129046.16208729 166529.11949459]
 [     0.          77491.64979611 100000.        ]
 [     0.              0.          60049.55788123]]
"""

[[100000.         129046.16208729 166529.11949459]
 [     0.          77491.64979611 100000.        ]
 [     0.              0.          60049.55788123]]


'\n[[100000.         129046.16208729 166529.11949459]\n [     0.          77491.64979611 100000.        ]\n [     0.              0.          60049.55788123]]\n'

### Step 5: GS Valuation

In [12]:
# decision_tee : 의사결정트리 
# gs_value_tree : gs 가치트리 
# holding_value_tree : gs가치트리에서 전 노드 보유가치

decision_tree = np.zeros_like(stock_binomial_tree)
discount_factor_tree = np.zeros_like(stock_binomial_tree)
gs_value_tree = np.zeros_like(stock_binomial_tree)
holding_value_tree = np.zeros_like(stock_binomial_tree)

# 가장 마지막 값을 업데이트
gs_value_tree[:, -1] = np.maximum(stock_binomial_tree[:, -1], bond_array[-1])

# 가장 마지막 값이 전환트리랑 똑같다면, 1, 아니면 0
decision_tree[:, -1] = np.where(gs_value_tree[:, -1] == stock_binomial_tree[:, -1], 1, 0)

# 가장 마지막의 할인율 업데이트
# if 전환: rf, if 상환: kd
discount_factor_tree[:, -1] = np.where(decision_tree[:, -1] == 1, rf, kd_forward_array[-1])

for t in reversed(range(gs_value_tree.shape[1] - 1)):

    # t시점의 holding value를 업데이트한다.

    holding_value_tree[:t+1, t] = (gs_value_tree[:t+1, t+1] * p * np.exp(-discount_factor_tree[:t+1,t+1]) 
                                   + gs_value_tree[1:t+2, t+1] * q * np.exp(-discount_factor_tree[1:t+2, t+1]))

    # t시점의 rcps value를 업데이트한다.
    gs_value_tree[:t+1, t] = np.maximum(np.maximum(np.maximum(stock_binomial_tree[:t+1, t], bond_array[t]), put_ammount_array[t]), holding_value_tree[:t+1, t])

    # t시점의 rcps value로 부터 상환 전환 보유 의사결정을 업데이트한다.
    decision_tree[:t+1, t] = np.where(gs_value_tree[:t+1, t] == stock_binomial_tree[:t+1, t], 1,
                                        np.where(gs_value_tree[:t+1, t] == put_ammount_array[t], 0,
                                                decision_tree[:t+1, t+1] * p + decision_tree[1:t+2, t+1] * q))

    # 의사결정트리에 discount factor tree를 업데이트한다.
    # if 전환: rf, if 상환: kd else: t+1시점의 discount factor * t+1시점의 decision_tree의 확률
    discount_factor_tree[:t+1, t] = np.where(decision_tree[:t+1, t] == 1, rf,
                                                np.where(decision_tree[:t+1, t] == 0, kd_forward_array[t],
                                                        (decision_tree[:t+1, t] * rf) + (
                                                            (1 - decision_tree[:t+1, t]) * kd_forward_array[t])
                                                        )
                                                )


In [13]:
gs_value_tree[:,0]

array([127819.6380825,      0.       ,      0.       ,      0.       ,
            0.       ,      0.       ,      0.       ,      0.       ,
            0.       ,      0.       ,      0.       ])

### Step 6: 엑셀로 결과물뽑기
위 코드를 엑셀과 같이 확인하기 위한 작업이다. 눈으로 직접 보면서, 계산을 따라갈 수 있는 엑셀처럼 결과값들을 한눈에 볼 수 있도록 엑셀을 구성한다.

- stock_binomial_tree : 주가이항과정
- put_ammount_array : 상환가능금액
- bond_array : 사채가치
- decision_tree : 의사결정트리 
- holding_value_tree : 보유가치트리 
- discount_factor_tree : 할인율트리
- gs_value_tree : tf 가치트리 

In [14]:
excel_data = {
"stock_binomial_tree" : stock_binomial_tree,
"put_ammount_array" : put_ammount_array,
"bond_array" : bond_array,
"decision_tree" : decision_tree,
"holding_value_tree" : holding_value_tree,
"discount_factor_tree" : discount_factor_tree,
"gs_value_tree" : gs_value_tree,
}

excel_path = "Ch2.1_GoldmanSachs.xlsx"

with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    # 데이터프레임을 엑셀 파일에 저장할 때, 시트 이름을 지정하여 새로운 시트를 추가합니다.
    for key in excel_data.keys():
        result_df = pd.DataFrame(excel_data[key])
        result_df.to_excel(
            writer, sheet_name=key, index=True)
